In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import re

In [2]:
def get_soup(url, params=None, headers=None):
   response = requests.get(url, params=params, headers=headers)
   soup = BeautifulSoup(response.content, "html.parser")
   return soup

In [16]:
def get_immobiliare_info(base_url):
    results = []

    for i in range(1, #funzione giuse): 
        url = base_url.format(i)
        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")

            announcement_elements = soup.find_all('div', class_='nd-mediaObject nd-mediaObject--colToRow in-reListCard')

            for element in announcement_elements:

                area_element = element.find('div', class_='in-reListCardFeatureList__item')
                area = area_element.find('span').text.strip() if area_element else None

                location_element = element.find('a', class_='in-reListCard__title')
                location = location_element.text.strip() if location_element else None

                price_element = element.find('div', class_='in-reListCardPrice')
                price = price_element.find('span').text.strip() if price_element else None

                results.append({
                    "area": area,
                    "location": location,
                    "price": price
                })

        else:
            print("Failed to retrieve the page:", url)

    return results

base_url = 'https://www.immobiliare.it/vendita-terreni/lombardia/?criterio=rilevanza&superficieMinima=20000&pag={}'
infos = get_immobiliare_info(base_url)
if infos:
    for info in infos:
        print(info)


TypeError: 'str' object cannot be interpreted as an integer